In [6]:
!pip install datasets

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [7]:
from datasets import load_dataset

dataset = load_dataset(
    "isek-ai/danbooru-tags-2016-2023",
    "all", # or "all"
    split="train",
)

In [4]:
print(dataset[23465]['rating'].split(','))

['s']


In [30]:
# Generate complete whitelist

data = dict()
count = 0

for temp in dataset:
	count += 1
	general = temp['general']
	character = temp['character']
	copyright = temp['copyright']
	artist = temp['artist']
	
	if(general == None):
		continue
		
	temp = general.split(',')
	
	if(character != None):
		temp += character.split(',')
		
	if(copyright != None):
		temp += copyright.split(',')
		
	if(artist != None):
		temp += artist.split(',')
		
	for tag in temp:
		tag = tag.strip()
		if(tag in data):
			data[tag] += 1
		else:
			data[tag] = 1

	if count % 100000 == 0:
		print(str(count / len(dataset) * 100) + "%")

2.1731774701476043%
4.346354940295209%
6.519532410442813%
8.692709880590417%
10.865887350738022%
13.039064820885626%
15.212242291033231%
17.385419761180835%
19.558597231328438%
21.731774701476045%
23.90495217162365%
26.078129641771252%
28.251307111918855%
30.424484582066462%
32.59766205221407%
34.77083952236167%
36.944016992509276%
39.117194462656876%
41.29037193280448%
43.46354940295209%
45.63672687309969%
47.8099043432473%
49.983081813394904%
52.156259283542504%
54.32943675369011%
56.50261422383771%
58.67579169398531%
60.848969164132924%
63.02214663428053%
65.19532410442814%
67.36850157457573%
69.54167904472334%
71.71485651487095%
73.88803398501855%
76.06121145516616%
78.23438892531375%
80.40756639546136%
82.58074386560897%
84.75392133575656%
86.92709880590418%
89.10027627605179%
91.27345374619938%
93.44663121634699%
95.6198086864946%
97.79298615664219%
99.96616362678981%


In [31]:
listed = list(data.items())
listed = sorted(listed, key=lambda x:x[1], reverse=True)

In [32]:
f = open("out/whitelist.txt", 'w')

for temp in listed:
    f.write(temp[0] + "|" + str(temp[1]) + "\n")
    
f.close()

In [4]:
## GENERATE tags.csv

f = open("out/tags.csv", 'w')

size = len(dataset)
for i in range(0, size):
  temp = dataset[i]
  prompt = ""

  artist = temp['artist']
  if artist != None:
    prompt += artist + ", "

  character = temp['character']
  if character != None:
    prompt += character + ", "
    
  copyright = temp['copyright']
  if copyright != None:
    prompt += copyright + ", "
    
  rating = temp['rating']
  if rating != None:
    prompt += "rating:" + rating + ", "
  else:
    prompt += "rating:none, "

  general = temp['general']
  if general != None:
    prompt += general + "\n"
    f.write(prompt)

  if i % 100000 == 0:
    print(str(i / size * 100) + "%")

f.close()

0.0%
2.227772842023281%
4.455545684046562%
6.683318526069844%
8.911091368093125%
11.138864210116406%
13.366637052139687%
15.594409894162967%
17.82218273618625%
20.04995557820953%
22.277728420232812%
24.505501262256093%
26.733274104279374%
28.961046946302655%
31.188819788325933%
33.416592630349214%
35.6443654723725%
37.872138314395784%
40.09991115641906%
42.32768399844234%
44.555456840465624%
46.7832296824889%
49.011002524512186%
51.238775366535464%
53.46654820855875%
55.69432105058203%
57.92209389260531%
60.14986673462859%
62.377639576651866%
64.60541241867516%
66.83318526069843%
69.06095810272171%
71.288730944745%
73.51650378676827%
75.74427662879157%
77.97204947081484%
80.19982231283812%
82.4275951548614%
84.65536799688468%
86.88314083890796%
89.11091368093125%
91.33868652295453%
93.5664593649778%
95.79423220700109%
98.02200504902437%


In [5]:
## Generate dictionary

size = len(dataset)
index = 0

positions = dict()

for i in range(0, size):
    temp = dataset[i]
    prompt = ''

    artist = temp['artist']
    if artist != None:
        prompt += artist + ', '

    character = temp['character']
    if character != None:
        prompt += character + ', '
        
    copyright = temp['copyright']
    if copyright != None:
        prompt += copyright + ", "
        
    rating = temp['rating']
    if rating != None:
        prompt += "rating:" + rating + ", "
    else:
        prompt += "rating:none, "

    general = temp['general']
    if general != None:
        prompt += general + "\n"

        tags = prompt.split(',')
        for tag in tags:
            tag = tag.strip()
            if positions.get(tag) == None:
                positions[tag] = list()
                
            positions[tag].append(index)
            
        index += len(prompt)

    if i % 100000 == 0:
        print(str(i / size * 100) + '%')

0.0%
2.227772842023281%
4.455545684046562%
6.683318526069844%
8.911091368093125%
11.138864210116406%
13.366637052139687%
15.594409894162967%
17.82218273618625%
20.04995557820953%
22.277728420232812%
24.505501262256093%
26.733274104279374%
28.961046946302655%
31.188819788325933%
33.416592630349214%
35.6443654723725%
37.872138314395784%
40.09991115641906%
42.32768399844234%
44.555456840465624%
46.7832296824889%
49.011002524512186%
51.238775366535464%
53.46654820855875%
55.69432105058203%
57.92209389260531%
60.14986673462859%
62.377639576651866%
64.60541241867516%
66.83318526069843%
69.06095810272171%
71.288730944745%
73.51650378676827%
75.74427662879157%
77.97204947081484%
80.19982231283812%
82.4275951548614%
84.65536799688468%
86.88314083890796%
89.11091368093125%
91.33868652295453%
93.5664593649778%
95.79423220700109%
98.02200504902437%


In [6]:
listedPos = list(positions.items())
listedPos = sorted(listedPos, key=lambda x:len(x[1]), reverse=True)

In [8]:
listedPos[3]

('rating:s',
 [11584,
  22216,
  48685,
  53830,
  56093,
  57548,
  60481,
  60823,
  62029,
  62801,
  74477,
  76421,
  79114,
  82883,
  84554,
  85759,
  88776,
  91940,
  92423,
  94017,
  94714,
  95549,
  97153,
  99160,
  104745,
  107565,
  110798,
  112191,
  113167,
  114485,
  121839,
  122169,
  124154,
  126994,
  127347,
  128515,
  134560,
  138846,
  141121,
  142693,
  144315,
  144977,
  145709,
  149160,
  150106,
  154747,
  156404,
  162660,
  164398,
  166741,
  168357,
  170454,
  171704,
  172834,
  173743,
  174610,
  178313,
  178697,
  180444,
  182603,
  186246,
  190365,
  194588,
  196171,
  201343,
  202945,
  204595,
  205036,
  205730,
  209533,
  210058,
  210909,
  211362,
  213666,
  216062,
  218318,
  219768,
  224363,
  225646,
  227748,
  229788,
  230520,
  236265,
  242285,
  242863,
  243740,
  244816,
  245211,
  249041,
  250345,
  252369,
  255850,
  257090,
  257406,
  267842,
  268269,
  270127,
  270703,
  272623,
  280508,
  283511,
 

In [9]:
## Generate pos.csv and key.csv

import binascii

pos = open('out/pos.csv', 'wb') ## Index of tags.csv
key = open('out/key.csv', 'w') ## Index of pos.csv

index = 0

for temp in listedPos:
    key.write(temp[0] + "|" + str(index) + "\n")
    
    for i in temp[1]:
        hexed = hex(i)[2:].zfill(8)
        pos.write(binascii.unhexlify(hexed))
        index += 1 ## MUST *4 TO GET INDEX
    
pos.close()
key.close()

In [13]:
artist_list = open('dataset/artist_list.txt', 'r')
character_list = open('dataset/character_list.txt', 'r')
characteristic_list = open('dataset/characteristic_list.txt', 'r')
copyright_list = open('dataset/copyright_list.txt', 'r')
censor_list = open('dataset/censor_list.txt', 'r')
whitelist = open('dataset/whitelist.txt', 'r')
out = open('dataset/out.txt', 'w')

artist_list = artist_list.read().split('\n')
character_list = character_list.read().split('\n')
censor_list = censor_list.read().split('\n')
whitelist = whitelist.read().split('\n')
copyright_list = copyright_list.read().split('\n')

# combine all lists without duplicates
combined_list = []

combined_list.extend(whitelist)
combined_list.extend(artist_list)
combined_list.extend(character_list)
combined_list.extend(copyright_list)
combined_list.extend(characteristic_list)
combined_list.extend(censor_list)

# remove empty strings

# remove duplicates
combined_list = list(dict.fromkeys(combined_list))

# write to file
for tag in combined_list:
	tag = tag.strip()
	out.write(tag + '\n')

out.close()

In [100]:
whitelist = open('dataset/whitelist.txt', 'r')
artist_list = open('dataset/artist_list.txt', 'r')
character_list = open('dataset/character_list.txt', 'r')
characteristic_list = open('dataset/characteristic_list.txt', 'r')
copyright_list = open('dataset/copyright_list.txt', 'r')
censor_list = open('dataset/censor_list.txt', 'r')
remove_list = open('out/removelist.txt', 'r')
clothes_list = open('dataset/clothes_list.txt', 'r')

data = []

whitelist = whitelist.read().split('\n')
artist_list = artist_list.read().split('\n')
character_list = character_list.read().split('\n')
characteristic_list = characteristic_list.read().split('\n')
copyright_list = copyright_list.read().split('\n')
censor_list = censor_list.read().split('\n')
remove_list = remove_list.read().split('\n')
clothes_list = clothes_list.read().split('\n')

for tag in whitelist:
	if tag in artist_list:
		continue
	if tag in character_list:
		continue
	if tag in characteristic_list:
		continue
	if tag in copyright_list:
		continue
	if tag in censor_list:
		continue
	if tag in remove_list:
		continue
	if tag in clothes_list:
		continue
	data.append(tag)


In [13]:
out = open('dataset/out.txt', 'w')

for tag in data:
	out.write(tag + '\n')
	

In [14]:
clothes_list = []

In [104]:
out = open('dataset/out.txt', 'w')
clothes_list = open('dataset/find.txt', 'r')
clothes_list = clothes_list.read().split('\n')

for tag in data:
	for tag2 in clothes_list:
		if (len(tag2) > len(tag)):
			continue
		if(tag[-len(tag2):] == tag2):
			out.write(tag + '\n')
			break

In [103]:
out = open('dataset/out.txt', 'w')
clothes_list = open('dataset/find.txt', 'r')
clothes_list = clothes_list.read().split('\n')

for tag in data:
	for tag2 in clothes_list:
		if tag2 in tag:
			out.write(tag + '\n')
			break

In [106]:
out = open('dataset/out.txt', 'w')
color = open('dataset/find.txt', 'r')
color = color.read().split('\n')

for tag in data:
	splited = tag.split(' ')

	for tag2 in color:
		if(splited[len(splited) - 1] == tag2):
			out.write(tag + '\n')
			break
		# if(tag2 in tag):
		# 	out.write(tag + '\n')
		# 	break


out.close()

In [12]:
## Generate dictionary

size = len(dataset)
index = 0

character = dict()

for i in range(0, size):
	temp = dataset[i]
	ch = temp['character']

	if ch != None and temp['general'] != None and temp['copyright'] != None:
		ch = ch.split(',')
		for c in ch:
			c = c.strip()
			if character.get(c) == None:
				character[c] = dict()

			tags = temp['general'] + ',' + temp['copyright']
			
			for tag in tags.split(','):
				tag = tag.strip()
				if character[c].get(tag) == None:
					character[c][tag] = 1
				else:
					character[c][tag] += 1

		index += 1

	if i % 100000 == 0:
		print(str(i / size * 100) + '%')

0.0%
2.1731774701476043%


KeyboardInterrupt: 

In [4]:
## GENERATE safetags.csv

f = open("out/safetags.csv", 'w')
tags = []

size = len(dataset)
for i in range(0, size):
	temp = dataset[i]

	rating = temp['rating']
	if rating == 'g':
		general = temp['general']
		if general != None:
			general = general.split(',')
			for tag in general:
				tag = tag.strip()
				if tag not in tags:
					tags.append(tag)

	if i % 100000 == 0:
		print(str(i / size * 100) + "%")

f.close()

0.0%
2.1731774701476043%
4.346354940295209%
6.519532410442813%
8.692709880590417%
10.865887350738022%
13.039064820885626%
15.212242291033231%
17.385419761180835%
19.558597231328438%
21.731774701476045%
23.90495217162365%
26.078129641771252%
28.251307111918855%
30.424484582066462%
32.59766205221407%
34.77083952236167%
36.944016992509276%
39.117194462656876%
41.29037193280448%
43.46354940295209%
45.63672687309969%
47.8099043432473%
49.983081813394904%
52.156259283542504%
54.32943675369011%
56.50261422383771%
58.67579169398531%
60.848969164132924%
63.02214663428053%
65.19532410442814%
67.36850157457573%
69.54167904472334%
71.71485651487095%
73.88803398501855%
76.06121145516616%
78.23438892531375%
80.40756639546136%
82.58074386560897%
84.75392133575656%
86.92709880590418%
89.10027627605179%
91.27345374619938%
93.44663121634699%
95.6198086864946%
97.79298615664219%
99.96616362678981%


In [5]:
f = open("out/safetags.csv", 'w')

for tag in tags:
	f.write(tag + '\n')

f.close()

In [8]:
fnsfw = open("out/nsfw.csv", 'w')
fsafe = open("out/safetags.csv", 'r')
fwhite = open("dataset/whitelist.txt", 'r')

safetags = fsafe.read().split('\n')
fsafe.close()

whitelist = fwhite.read().split('\n')
fwhite.close()

for tag in whitelist:
	if tag in safetags:
		continue

	fnsfw.write(tag + '\n')

fnsfw.close()

